## 1. 개발자 노트 댓글 수집

In [1]:
import pandas as pd 
from bs4 import BeautifulSoup 
from selenium import webdriver 
from selenium.webdriver.common.by import By
from collections import defaultdict
from tqdm import tqdm 
import time 
import re

In [9]:
driver = webdriver.Chrome("C:/Users/park/Desktop/Neople/chromedriver.exe")
driver.get('https://df.nexon.com/df/mediazone/magazine/devnote')

C:\Users\park\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [13]:
contents = driver.find_element(by=By.XPATH, value = '//*[@id="container"]/div[2]/div[2]/div[1]')
before_url = contents.find_elements(by = By.XPATH, value = '//*[@id="container"]/div[2]/div[2]/div[1]/ul/li/dl/dt/a')
video_info_after = list(map(lambda x, y: (x.text, y.get_attribute('href')), before_url, before_url))

In [15]:
url_dataset = pd.DataFrame(video_info_after, columns = ["title", "url"])
url_dataset

,title,url
0,4월 업데이트와 진행 상황에 대해 말씀드립니다,https://df.nexon.com/df/mediazone/magazine/dev...
1,최고 레벨 확장의 방향성에 대해 말씀드립니다,https://df.nexon.com/df/mediazone/magazine/dev...
2,던전앤파이터 개발 방향성에 대해 말씀드립니다.,https://df.nexon.com/df/mediazone/magazine/dev...
3,새해 첫인사 드립니다,https://df.nexon.com/df/mediazone/magazine/dev...
4,안녕하세요. 던전앤파이터 윤명진입니다.,https://df.nexon.com/df/mediazone/magazine/dev...
5,11월 업데이트 관련 안내,https://df.nexon.com/df/mediazone/magazine/dev...
6,엠블렘 시스템 개편과 이후 방향에 대해서,https://df.nexon.com/df/mediazone/magazine/dev...


In [147]:
url_dataset.iloc[:3,:].values[-1]

array(['던전앤파이터 개발 방향성에 대해 말씀드립니다.',
       'https://df.nexon.com/df/mediazone/magazine/devnote?p=web&mode=view&no=2622261'],
      dtype=object)

In [148]:
title_comment_df = pd.DataFrame()
for title, url in tqdm([url_dataset.iloc[:3,:].values[-1]]):
    driver.get(url)
    total_list = []
    while True:
        comment_contents = driver.find_element(by=By.XPATH, value = '//*[@id="reply"]/ul')
        id_server_list = comment_contents.find_elements(by=By.XPATH, value = '//*[@id="reply"]/ul/li/p[1]/span[2]')
        date_list = comment_contents.find_elements(by=By.XPATH, value = '//*[@id="reply"]/ul/li/p[1]/span[3]')
        comment_list = comment_contents.find_elements(by=By.XPATH, value = '//*[@id="reply"]/ul/li/p[2]')
        id_date_comment_list = list(map(lambda x, y, z : x.text.split(" |") + [y.text, z.text],id_server_list, date_list, comment_list))
        total_list += id_date_comment_list

        current_url = driver.current_url
        time.sleep(1)
        driver.find_element(by=By.XPATH, value='//*[@id="reply"]/div[2]/a[2]').click()
        after_click_url = driver.current_url
        if current_url == after_click_url:
            break

            
    total_df = pd.DataFrame(total_list, columns=["user_id", "server", "date", "comment"])        
    total_df["title"] = title
    title_comment_df = pd.concat([title_comment_df, total_df], axis=0)    

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [03:03<00:00, 183.33s/it]


In [150]:
title_comment_df.to_csv("./던전엔파이터개발방향성에대해말씀드립니다.csv")

## 2. '던파나우' 크리쳐 시세 데이터 수집

In [2]:
driver = webdriver.Chrome("C:/Users/park/Desktop/Neople/chromedriver.exe")
driver.get('http://dnfnow.xyz/item?item_idx=4b358426ff59ed138a521b47a0b5cc06')

C:\Users\park\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [17]:
driver.find_element(by=By.XPATH, value = '//*[@id="day"]/div').get_attribute()

TypeError: get_attribute() missing 1 required positional argument: 'name'

## 3. '던파OFF' 결전, 근원 무기 채용 데이터 수집

In [18]:
driver = webdriver.Chrome("C:/Users/park/Desktop/Neople/chromedriver.exe")
driver.get('https://dunfaoff.com/equip_chart.df')

C:\Users\park\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [124]:
idx = 0
job_weapon_dict = defaultdict()
job_xpath = '//*[@id="select_div"]/select'
job_select = job_xpath + f"/option[{idx + 1}]"
job_name_list = driver.find_element(by=By.XPATH, value = job_xpath).text

job_list = list(map(lambda x : x.strip(), job_name_list.split("\n")))
job_list = [i for i in job_list if len(i) > 1]
job_list[:5]

['검신', '다크로드', '블러드 이블', '인다라천', '악귀나찰']

In [125]:
while True:
    try:
        job_name = job_list[idx]
    except IndexError:
        break
    job_select = job_xpath + f"/option[{idx + 1}]"
    driver.find_element(by=By.XPATH, value = job_select).click()
    time.sleep(1)
    driver.find_element(by=By.XPATH, value = '//*[@id="searchbtn"]').click()
    time.sleep(1)

    body = driver.find_element(by=By.XPATH, value = '//*[@id="back"]/div/div[2]')
    weapons = body.find_elements(by=By.XPATH, value = '//*[@id="back"]/div/div[2]/div/div')
    tmp = [i.text for i in weapons][2:]

    job_weapon_dict[job_name] = tmp
    idx += 1
job_weapon_dict = dict(job_weapon_dict)    

In [192]:
job_weapon_df = pd.DataFrame()
for job in job_weapon_dict:
    tmp_df = pd.DataFrame(job_weapon_dict[job], columns=["weapon"])
    tmp_df["job"] = job
    job_weapon_df = job_weapon_df.append(tmp_df)
    
weapon_105 = job_weapon_df[job_weapon_df["weapon"].str.contains("근원을|결전의")]    
weapon_105["weapon_name"] = weapon_105["weapon"].apply(lambda x : x.split("\n")[0])
weapon_105["weapon_ratio"] = weapon_105["weapon"].apply(lambda x : x.split("\n")[1])
weapon_105 = weapon_105.reset_index(drop=True)

weapon_105["weapon_count"] = weapon_105["weapon_ratio"].apply(lambda x : x.split()[0])
weapon_105["weapon_percent"] = weapon_105["weapon_ratio"].apply(lambda x: x.split()[1])
weapon_105["weapon_count"] = weapon_105["weapon_count"].apply(lambda x: int(re.sub(",","",x)))

C:\Users\park\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\park\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [220]:
weapon_105.to_csv("./20220426_weapon_count.csv")

In [221]:
tmp = []
for name in weapon_105["weapon_name"]:
    if "결전의" in name:
        tmp += ["결전 무기"]
    else:
        tmp += ["근원 무기"]
weapon_105["new_name"] = tmp
result = weapon_105["weapon_count"]
result.index = weapon_105["new_name"] + " " + weapon_105["job"]

In [225]:
result = pd.DataFrame(result.sum(level=0))
result

,weapon_count
근원 무기 검신,6269
결전 무기 검신,2354
결전 무기 다크로드,3886
근원 무기 다크로드,1552
결전 무기 블러드 이블,4826
...,...
결전 무기 갓파더,834
근원 무기 패스파인더,320
결전 무기 패스파인더,221
근원 무기 레퀴엠,1703
